In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')

In [4]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [41]:
pd.read_sql(
    """
    select 
    count (*) filter (where CRIM=0.0) as CRIM,
    count(*) filter (where zn=0.0) as ZN,
    count (*) filter (where INDUS=0.0) as INDUS,
    count (*) filter (where CHAS=0.0) as CHAS,
    count (*) filter (where NOX=0.0) as NOX from boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,372,0,471,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [42]:
pd.read_sql(
    """
    select count(distinct CRIM) filter (where CRIM!=0.0) as CRIM,
    count (distinct ZN) filter (where ZN!=0.0) as ZN,
    count (distinct INDUS) filter (where INDUS!=0.0) as INDUS,
    count (distinct CHAS) filter (where CHAS!=0.0) as CHAS,
    count (distinct NOX) filter (where NOX!=0.0) as NOX
    from boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,25,76,1,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [45]:
pd.read_sql(
    """
    select CRIM, ZN, INDUS, CHAS, NOX from boston LIMIT 1 offset (select count(*) from boston) /2
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0.36894,22.0,5.86,0.0,0.431


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех знаечений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)